# 1 - Importa bibliotecas

In [1]:
import os
import gym
import numpy as np
import torch as th
import websocket
import json
import time
import random
import psutil

# 2 - Cria funções de encoding/decoding em json

In [2]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)                
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
def encode_json(identifier, reward, done, states):
    data = {}
    data['identifier'] = identifier
    data['reward'] = reward
    data['done'] = done
    data['states'] = states
    json_data = json.dumps(data, cls=NumpyEncoder)       
    return json_data

In [3]:
def enviroment(data, reward, done,step, CountCan,correctCan, ConveyorPosition, Forward, Backward, RobotCycle, StartCycle, arrayPosicoes, ToDropCan):
        step += 1
        message = ''
        BoxCompleted = False
        Can = False
        SensorHandling = False
        SensorRobot = False
        CanDropped = False
        
        #print(data)
        
        if data['actions'][0] == 0:
            Forward = False
            Backward = False
            
        if data['actions'][0] == 1:
            Forward = True
            Backward = False
                       
        if data['actions'][0] == 2:
            Backward = True
            Forward = False
        
        reward = 0
            
        if Forward:
            ConveyorPosition = ConveyorPosition + 100 
            
        if Backward:
            ConveyorPosition = ConveyorPosition - 100
            
        if ConveyorPosition < 20 and RobotCycle == 0:
            SensorRobot = True 
        if ConveyorPosition > 342 and ConveyorPosition < 700:
            SensorHandling = True
        if ConveyorPosition < 342 and ConveyorPosition > 700:
            SensorHandling = False
        
        if correctCan == 9:
            BoxCompleted == True
            #print('--- passo: ', step, 'posicoes de lata: ', arrayPosicoes)
        if correctCan != 9: 
            BoxCompleted == False
            
        if  SensorHandling and not BoxCompleted and not Forward and not Backward:            
            CountCan += 1
            arrayPosicoes.append(ConveyorPosition)
            CanDropped = True
            if CountCan % 3 == 0:
                Can = True
                
                      
        if ConveyorPosition > 579 and ConveyorPosition < 601 and CountCan < 10 and CountCan > 6 and CanDropped:
            reward = 300 - step/5
            correctCan += 1
        if ConveyorPosition > 479 and ConveyorPosition < 501 and CountCan < 7 and CountCan > 3 and CanDropped:
            reward = 200 - step/5
            correctCan += 1
        if ConveyorPosition > 379 and ConveyorPosition < 401 and CountCan < 4 and CanDropped:
            reward = 100 - step/5
            correctCan += 1
            
            
        
        if RobotCycle > 0:
            if RobotCycle > 1:
                reward = 1000 + (correctCan*1000)
                print('-------------------Aguardando!------------------------- passo: ', step)
            if RobotCycle == 1:
                done = True
                reward = 10000 + (correctCan  * 7000) - step
                print('-------------------Ciclo completo!------------------------- passo: ', step)
            RobotCycle -= 1 
        
        if RobotCycle < 3 and RobotCycle > 0:
            SensorRobot = False
        
        if SensorRobot and StartCycle:
            RobotCycle = random.randint(3, 5)
            StartCycle = False
            reward = 1000 + (correctCan  * 150)
            CountCan == 0
            print('Aguardando descarga------------')
        if ConveyorPosition > 0 and correctCan == 9 and Backward:
            reward = (correctCan * 120)  - (ConveyorPosition/8)                
            print('<<<<<----- voltando ---<<<<<    posicao: ',ConveyorPosition,' numero de latas corretas: ', correctCan, ' recompensa: ',reward, )
        
        if ConveyorPosition < 20 and ConveyorPosition > -1 and correctCan == 9 and not StartCycle and not Forward and not Backward:
            reward = 5000 + (correctCan * 2000) 
            SensorRobot = True
            StartCycle = True
            print('@@@@@-------pronto a descarregar!@@@@')
                
            
            
        
        #if CountCan > 9:
            #reward = 100 - (step *(11 - CountCan))
            #if CountCan > 12:
                #done = True
        
            
        #if ConveyorPosition < 0:
            #reward = ConveyorPosition*10
            
        if ConveyorPosition < -200 or ConveyorPosition > 800:
            done = True
        
        if reward > 90 and not Backward and not Forward:
            print('passo: ',step,' recompensa: ',reward, ' posicoes de lata: ', arrayPosicoes,' posicao: ', ConveyorPosition )
            
        states = [Can, SensorHandling, SensorRobot, round((ConveyorPosition/600),2)]
        if CountCan > 15:
            pass
            #done = True
            
        if step > 500:            
            done = True
        
        message = encode_json('BoxConveyor', reward, done, states)    
        return message, reward, done,step, CountCan,correctCan, ConveyorPosition, Forward, Backward, RobotCycle, StartCycle, arrayPosicoes, ToDropCan

# 4 - Verifica se o servidor socket está ativo

In [4]:
server_found = False

for conn in psutil.net_connections():
    if conn.laddr.port == 12000 and conn.status == 'LISTEN':
        print("Um servidor socket está sendo executado na porta 12000.")
        server_found = True

if not server_found:
    print("Nenhum servidor socket está sendo executado na porta 12000.")

Um servidor socket está sendo executado na porta 12000.


# 5 - Ativa o ambiente para treino

In [9]:
ws = websocket.WebSocket()
ws.connect("ws://127.0.0.1:12000")
time.sleep(1)
reward = 0
done = False
CountCan = 0
ConveyorPosition = 0
Forward = False
Backward = False
StartCycle = False
step = 0
RobotCycle = 0
message = ''
arrayPosicoes = []
ToDropCan = 0
correctCan = 0

while True:
    data =''
    data = json.loads(ws.recv())
    if data['identifier']  == 'BoxConveyor_manual':
        message, reward, done,step, CountCan,correctCan, ConveyorPosition, Forward, Backward, RobotCycle, StartCycle, arrayPosicoes, ToDropCan = enviroment(data, reward, done, step, CountCan, correctCan,ConveyorPosition, Forward, Backward,RobotCycle,StartCycle, arrayPosicoes, ToDropCan) 
        ws.send(message)
        if done:
            done = False
            reward = 0
            step = 0
            CountCan = 0
            ConveyorPosition = 0
            Forward = False
            Backward = False
            RobotCycle = 0
            StartCycle = False
            arrayPosicoes = []
            ToDropCan =0
            correctCan = 0

ConnectionResetError: [WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto